In [1]:
# pip install qiskit==0.31.0

In [2]:
from get_test_datasets import return_dataset
test_dataset = return_dataset()

/home/ritu/anaconda3/envs/dnn_for_qubit_mapping/lib/python3.8/site-packages/qiskit/version.py:20: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


Generating QPE circuits


In [3]:
len(test_dataset)

400

In [4]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from helper import *

Using TensorFlow backend.
ibmqfactory.load_account:WARNING:2023-11-27 11:00:18,393: Credentials are already in use. The existing account in the session will be replaced.


{'ibm_lagos': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]], 'ibm_nairobi': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]], 'ibm_perth': [[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]]}


In [5]:
mlp_edge_feature = tf.keras.models.load_model("models/mlp_edge_features.keras")
dnn = tf.keras.models.load_model("models/dnn.keras")
mlp = tf.keras.models.load_model("models/mlp.keras")

2023-11-27 11:00:21.664828: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-11-27 11:00:21.710942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-27 11:00:21.711287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4090 computeCapability: 8.9
coreClock: 2.595GHz coreCount: 128 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 938.86GiB/s
2023-11-27 11:00:21.711621: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-11-27 11:00:21.711752: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic lib

In [6]:
basis_gate_dict = {}
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends(simulator=False)
backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
for backend_name in backends:
    backend = provider.get_backend(backend_name)
    basis_gats = backend.configuration().basis_gates
    basis_gate_dict[backend_name] = basis_gats


In [7]:
def get_transpiled_circ_results(qc, backend, initial_layout):
    basis_gats = basis_gate_dict[backend]
    pass_ = Unroller(basis_gats)
    pm = PassManager(pass_)
    backend = provider.get_backend(backend)
    
    tqc = transpile(qc, backend=backend,initial_layout=initial_layout, optimization_level=0,seed_transpiler=13)
    tqc = pm.run(tqc)
    gates = dict(tqc.count_ops())
    return gates['cx'], tqc.depth()


In [19]:
SS = StandardScaler()
num_qubits = 7
count_or = 0
count_nr = 0
count_mlp = 0
count_mlp_edge = 0
original_depth, original_CNOT = [],[]
mlp_edge_depth, mlp_edge_CNOT = [],[]
dnn_without_ro_depth,dnn_without_ro_CNOT = [],[] 
dnn_with_ro_depth,dnn_with_ro_CNOT = [],[] 
mlp_depth, mlp_CNOT = [],[]

dnn_time = []
mlp_time = []
mlp_edge_time = []
sabre_time,na_time,dense_time = [],[],[]
for qc_i in tqdm(range(len(test_dataset[-5:]))):
    qc = test_dataset[qc_i]
    data = datetime.today() - timedelta(days=random.randint(1,150))
    backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
    backend = backends[np.random.randint(0,3)]
    l = add_line(qc,backend, optimization_level=3, refresh=True, show= False, datatime=data)
    
    d={}
    for i in range(len(l[0])):
        d[str(l[0][i])] = l[1][i]
    df = pd.DataFrame(d, index=[0])
    df = clear_dataset(df, 7)
    last_num_qubits = len(df.columns)-num_qubits
    labels = ((df.iloc[:, last_num_qubits:].values)[0]).tolist()
    
#     time_taken_to_get_df = get_df_time(qc,backend,data)
#     print("Preprocess time:",time_taken_to_get_df)
    time_taken_to_get_df = 0
    
    start = time.time()
    x = SS.fit_transform(df.iloc[:, 2:last_num_qubits].values)
    predicted = dnn.predict(x)
    pred_or = np.array(pred_layout(predicted, num_qubits))[0].tolist()
    pred_nr = np.array(pred_layout_diff_elem(predicted,num_qubits))[0].tolist()
    end = time.time()
    dnn_time.append(time_taken_to_get_df + (end-start))
    
    print(pred_or, pred_nr, labels)
    if(pred_or == labels):
        count_or = count_or + 1
    if(pred_nr == labels):
        count_nr = count_nr + 1
    
    start = time.time()
    mlp_pred = get_labels(np.reshape(mlp.predict(x),(7,7)))
    end = time.time()
    mlp_time.append(time_taken_to_get_df + (end-start))
    print(mlp_pred)
    if mlp_pred == labels:
        count_mlp = count_mlp + 1
        
    start = time.time()
    x = SS.fit_transform(get_graph_features(df))
    mlp_edges_pred = get_labels(np.reshape(mlp_edge_feature.predict(x),(7,7)))
    end = time.time()
    mlp_edge_time.append(time_taken_to_get_df + (end-start))
    print(mlp_edges_pred)
    if mlp_edges_pred == labels:
        count_mlp_edge = count_mlp_edge + 1
    
    
    # original circuit after unrolling
    cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=labels)
    original_depth.append(depth)
    original_CNOT.append(cx)
    print(original_depth[-1],original_CNOT[-1])
    # NN with edge features
    if len(mlp_edges_pred) == len(set(mlp_edges_pred)):
        cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=mlp_edges_pred)
        mlp_edge_depth.append(depth)
        mlp_edge_CNOT.append(cx)
    else:
        mlp_edge_depth.append(float('NaN'))
        mlp_edge_CNOT.append(float('NaN'))
        
    # NN without edge features
    if len(mlp_pred) == len(set(mlp_pred)):
        cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=mlp_pred)
        mlp_depth.append(depth)
        mlp_CNOT.append(cx)
    else:
        mlp_depth.append(float('NaN'))
        mlp_CNOT.append(float('NaN'))
        
    # DNN without repair operator
    if len(pred_or) == len(set(pred_or)):
        cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=pred_or)
        dnn_without_ro_depth.append(depth)
        dnn_without_ro_CNOT.append(cx)
    else:
        dnn_without_ro_depth.append(float("NaN"))
        dnn_without_ro_CNOT.append(float("NaN"))
        
    # DNN with RO
    cx, depth = get_transpiled_circ_results(qc, backend, initial_layout=pred_nr)
    dnn_with_ro_depth.append(depth)
    dnn_with_ro_CNOT.append(cx)
    
#     # execution times
    basis_gats = basis_gate_dict[backend]
    pass_ = Unroller(basis_gats)
    pm = PassManager(pass_)
    backend = provider.get_backend(backend)
    tqc = transpile(qc, backend=backend, optimization_level=0)
    tqc = pm.run(tqc)
    # noise adaptive time
    start = time.time()
    new_circ_lv3_na = transpile(tqc, backend=backend, optimization_level=0,layout_method='noise_adaptive')
    new_circ_lv3_na._layout.get_physical_bits()
    end = time.time()
    na_time.append(end-start)
    # sabre time
    start = time.time()
    new_circ_lv3_sabre = transpile(tqc, backend=backend, optimization_level=0,layout_method='sabre',routing_method='sabre')
    new_circ_lv3_sabre._layout.get_physical_bits()
    end = time.time()
    sabre_time.append(end-start)
    # dense time
    start = time.time()
    new_circ_lv3_dense = transpile(tqc, backend=backend, optimization_level=0)
    new_circ_lv3_dense._layout.get_physical_bits()
    end = time.time()
    dense_time.append(end-start)
    

print("DNN accuracy without repair operator:", count_or/len(test_dataset))
print("DNN accuracy with repair operator:", count_nr/len(test_dataset))
print("MLP accuracy:", count_mlp/len(test_dataset))
print("MLP with edge features accuracy:", count_mlp_edge/len(test_dataset))

  0%|                                                                                                                                                            | 0/5 [00:00<?, ?it/s]

2023-06-30 11:22:39.558970
Backend topology: 2023-06-30 11:22:39.558970
Depth: [84, 84, 78, 78, 78, 78]
not na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



 20%|█████████████████████████████▌                                                                                                                      | 1/5 [00:02<00:10,  2.59s/it]

[2, 1, 0, 3, 4, 5, 6]
60 32
2023-08-20 11:22:42.150935
Backend topology: 2023-08-20 11:22:42.150935
Depth: [206, 206, 86, 86, 86, 86]
not na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [0, 1, 2, 3, 4, 5, 6]
[6, 1, 4, 3, 4, 5, 6]



 40%|███████████████████████████████████████████████████████████▏                                                                                        | 2/5 [00:04<00:07,  2.40s/it]

...

[2, 1, 0, 3, 4, 5, 6]
57 38
2023-10-28 11:22:46.429198
Backend topology: 2023-10-28 11:22:46.429198
Depth: [72, 72, 80, 80, 102, 102]
na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [4, 3, 5, 6, 0, 1, 2]
[6, 1, 4, 3, 4, 5, 6]



 80%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                             | 4/5 [00:08<00:02,  2.06s/it]

[2, 1, 0, 3, 4, 5, 6]
66 44
2023-09-26 11:22:48.232514
Backend topology: 2023-09-26 11:22:48.232514
Depth: [86, 86, 88, 88, 98, 98]
na
[2, 1, 0, 3, 4, 5, 6] [2, 1, 0, 3, 4, 5, 6] [4, 5, 6, 3, 2, 1, 0]
[6, 1, 4, 3, 4, 5, 6]



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:10<00:00,  2.08s/it]

[2, 1, 0, 3, 4, 5, 6]
56 29
DNN accuracy without repair operator: 0.0
DNN accuracy with repair operator: 0.0
MLP accuracy: 0.0
MLP with edge features accuracy: 0.0


In [20]:
df = pd.DataFrame()
df["Label Depth"] = original_depth
df["Label CNOTs"] = original_CNOT

df["NN without edge features Depth"] = mlp_depth
df["NN without edge features CNOTs"] = mlp_CNOT

df["NN with edge features Depth"] = mlp_edge_depth
df["NN with edge features CNOTs"] = mlp_edge_CNOT

df["DNN without repair Depth"] = dnn_without_ro_depth
df["DNN without repair CNOTs"] = dnn_without_ro_CNOT

df["DNN with repair Depth"] = dnn_with_ro_depth
df["DNN with repair CNOTs"] = dnn_with_ro_CNOT

df["DNN Mapping time"] = dnn_time
df["NN without edge features Mapping time"] = mlp_time
df["NN with edge features Mapping time"] = mlp_edge_time
df["SABRE mapping Mapping time"] = sabre_time
df["Noise Adaptive Mapping time"] = na_time
df["Dense Mapping time"] = dense_time

In [21]:
df

,Label Depth,Label CNOTs,NN without edge features Depth,NN without edge features CNOTs,NN with edge features Depth,NN with edge features CNOTs,DNN without repair Depth,DNN without repair CNOTs,DNN with repair Depth,DNN with repair CNOTs,DNN Mapping time,NN without edge features Mapping time,NN with edge features Mapping time,SABRE mapping Mapping time,Noise Adaptive Mapping time,Dense Mapping time
0,60,32,NaN,NaN,62,35,62,35,62,35,0.010940,0.008420,0.013839,0.011827,0.011510,0.002655
1,59,44,NaN,NaN,59,44,59,44,59,44,0.010616,0.008597,0.012853,0.014319,0.014587,0.002804
2,57,38,NaN,NaN,57,38,57,38,57,38,0.011061,0.008651,0.012619,0.009130,0.015255,0.002752
3,66,44,NaN,NaN,56,29,56,29,56,29,0.010974,0.008689,0.013744,0.012737,0.011660,0.002606
4,56,29,NaN,NaN,62,35,62,35,62,35,0.010890,0.008564,0.013134,0.008358,0.002935,0.002746


In [ ]:
import itertools
layouts = list(itertools.permutations([0,1,2,3,4,5,6]))
qc = test_dataset[-8]
basis_gats = basis_gate_dict['ibm_nairobi']
pass_ = Unroller(basis_gats)
pm = PassManager(pass_)
backend = provider.get_backend('ibm_nairobi')
for l in layouts[:20]:
    tqc = transpile(qc, backend=backend,initial_layout=list(l), optimization_level=0)
    tqc = pm.run(tqc)
    
    print(l,tqc.depth(),dict(tqc.count_ops())['cx'])

In [57]:
# def get_df(circuit,backend_name,datatime,show=False,refresh=True):
#     provider = IBMQ.get_provider(hub='ibm-q')
#     provider.backends(simulator=False)
#     backend = provider.get_backend(backend_name)
#     basis_gats = backend.configuration().basis_gates
#     # print(basis_gats)
#     pass_ = Unroller(basis_gats)
#     pm = PassManager(pass_)

#     # new_circ = pm.run(circuit)
#     # need to transpile before passing to run
#     new_circ = transpile(circuit, backend=backend, optimization_level=0)
#     new_circ = pm.run(new_circ)
    
#     size_backend = len(backend.properties(refresh=refresh).to_dict()['qubits'])
#     CA = circuit_analysis(backend, circuit, size_backend=size_backend, show=show)
#     BT = Backend_Topology(backend_name, refresh, show, datatime=datatime)
#     QP = Qubit_properties(backend_name, refresh, datatime=datatime)
    
#     Title_names =['last_update_date', 'backend_name'] + list(CA.keys())[:3] + list(CA['cx'].keys()) + list(CA['measure'].keys()) + list(BT['coupling'].keys())
#     for i in range(size_backend):
#         for title in list(QP.keys())[2:]:
#             Title_names.append(title+'_'+str(i))

#     date_name = [BT['last_update_date'], BT['backend_name']]
#     Values = date_name + list(CA.values())[:3] + list(CA['cx'].values()) + list(CA['measure'].values()) + list(BT['coupling'].values())
#     for i in range(size_backend):
#         for value in list(QP.keys())[2:]:
#             Values.append(QP[value][i])
#     return [Title_names,Values]

In [58]:
# qc = test_dataset[-5]

In [59]:
# l=get_df(qc,'ibm_nairobi',datetime.today(),show=False)

In [60]:
# l

In [61]:
# len(l[0]), len(l[1])

In [62]:

# df